# Purpose

The purpose of this notebook is to turn the keywords gathered in the protein table into features for the train set.

Lets write the "learning ready" features I'm actually going to use in the model into another sqlite database.

That way I can keep my "raw" data separate from my "prepared" data.

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
from tqdm import tqdm_notebook

In [3]:
conn = sqlite3.connect("human_protein.db")
cur = conn.cursor()
tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;").fetchall()
print(tables)
protein_df = pd.read_sql('SELECT * FROM protein',con=conn)
conn.close()
display(protein_df.head())

[('amy_ignore',), ('amyloid',), ('amyloid_prion',), ('amyloid_prion_id',), ('amyloid_prion_id_mapped',), ('protein',), ('protein_amino_acid_modifications',), ('protein_comments',), ('protein_diseases',), ('protein_keywords',), ('protein_secondary_structure',), ('protein_subcellular_localization',), ('protein_tissue_expression',)]


,protein,accession,sequence,keywords
0,RL37A_HUMAN,"P61513,P12751,Q6FGF5",MAKRTKKVGIVGKYGTRYGASLRKMVKKIEISQHAKYTCSFCGKTK...,"3D-structure, Complete proteome, Metal-binding..."
1,PYRG1_HUMAN,"P17812,B4DR64,D3DPW1,Q5VW67,Q96GK6",MKYILVTGGVISGIGKGIIASSVGTILKSCGLHVTSIKIDPYINID...,"3D-structure, Acetylation, Alternative splicin..."
2,RL6_HUMAN,"Q02878,Q2M3Q3,Q8WW97",MAGEKVEKPDTKEKKPEAKKVDAGGKVKKGNLKAKKPKKGKPHCSR...,"3D-structure, Acetylation, Complete proteome, ..."
3,RAB10_HUMAN,"P61026,D6W538,O88386,Q6IA52,Q9D7X6,Q9H0T3",MAKKTYDLLFKLLLIGDSGVGKTCVLFRFSDDAFNTTFISTIGIDF...,"3D-structure, Acetylation, Cell projection, Co..."
4,RAB30_HUMAN,"Q15771,Q6FGK1,Q6MZH2,Q96CI8",MSMEDYDFLFKIVLIGNAGVGKTCLVRRFTQGLFPPGQGATIGVDF...,"3D-structure, Alternative splicing, Complete p..."


In [4]:
# Parse keywords to columns
protein_df.loc[:,'keyword_split'] = protein_df['keywords'].str.split(',').apply(frozenset)
for keyword in tqdm_notebook(frozenset.union(*protein_df.keyword_split)):
    protein_df[keyword.strip()] = protein_df.apply(lambda x: int(keyword in x.keyword_split),axis=1)

In [6]:
protein_df_keywords = protein_df.drop(["keywords","keyword_split"],axis=1)
protein_df_keywords.drop(['accession','sequence'],axis=1,inplace=True)
protein_df_keywords.head()

,protein,,Helicase,Glycoprotein,Glycogen metabolism,Lectin,Chemotaxis,Fertilization,Antiport,B-cell activation,...,Inhibition of host RLR pathway by virus,Cysteine biosynthesis,Pheromone response,Endosome,rRNA-binding,Potassium channel,Parkinson disease,Viral immunoevasion,Phosphate transport,SH3 domain
0,RL37A_HUMAN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PYRG1_HUMAN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,RL6_HUMAN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,RAB10_HUMAN,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,RAB30_HUMAN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
protein_df_keywords_no_empty = protein_df_keywords[[x for x in protein_df_keywords.columns if len(x)>0]]

In [15]:
conn_train = sqlite3.connect("protein_training.db")
protein_df_keywords_no_empty.to_sql('protein_keywords', con=conn_train, index=False, if_exists='replace')
conn_train.close()

C:\Users\Jake\Anaconda3\lib\site-packages\pandas\core\generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [ ]:
#disease_df = pd.read_sql('SELECT * FROM protein_diseases',con=conn)
#display(disease_df.head())
#disease_df.groupby('disease').count().head()

Judging by how sparse disease is - I don't think it will contain enough information without some sort of parsing to be useful. Skip for now.